# Bioactive molecular networking notebook v1.1 (MZmine)

Website: https://github.com/DorresteinLaboratory/Bioactive_Molecular_Networks

The bioactive molecular network workflow integrates MS/MS molecular networking and bioassay-guided fractionation into the concept of bioactive molecular networking. The workflow relies on open bioinformatic tools, such MZmine2 [http://mzmine.github.io/] or Optimus (using OpenMS) [https://github.com/MolecularCartography/Optimus], a Jupyter notebook, and the GNPS web-platform (http://gnps.ucsd.edu). 

The code is released as a Jupyter notebook for easiness and reproducibility. The Jupyter notebook has been prepared by Dr. Ricardo Silva (UCSD).

#### Citation: 
Bioactive molecular networking: Nothias, L.-F.; Nothias-Esposito, M.; da Silva, R.; Wang, M.; Protsyuk, I.; Zhang, Z.; Sarvepalli, A.; Leyssen, P.; Touboul, D.; Costa, J.; Paolini J., Alexandrov T., Litaudon M., Dorrestein, P. Bioactivity-Based Molecular Networking for the Discovery of Drug Leads in Natural Product Bioassay-Guided Fractionation. J. Nat. Prod. 2018. 
#### Manuscript: 
(Open access) https://pubs.acs.org/doi/10.1021/acs.jnatprod.7b00737

### Instructions to run the notebook.

-> This notebook is suitable for MZmine feature table.

-> Make sure R is installed in your environment.

-> Upload your files to the jupyter notebook folder.

-> Update the filename if needed (in red) and indicated in the cell comments

-> Run all cells by clicking on: Cell / Run All Below.

-> Get the output file.

## Lets run the bioactive molecular networking notebook !

In [1]:
# Load and inspect the MZmine feature table with bioassay results
# Change the name in the code below if needed (.CSV file in red)
# NB: Make sure to add the value of bioactivity in the second row.
in_tab <- read.csv("TEMPLATE_FILES//INPUT_FILE/features_quantification_matrix_edited_bioactivity_MZmine2.csv", stringsAsFactor=FALSE, check.names=FALSE)
dim(in_tab)
in_tab[1:5,]

[1] 588  17

row ID,row m/z,row retention time,Extract.mzXML Peak area,F_5.mzXML Peak area,F_6.mzXML Peak area,F_7.mzXML Peak area,F_8.mzXML Peak area,F_9.mzXML Peak area,F_10.mzXML Peak area,F_11.mzXML Peak area,F_12.mzXML Peak area,F_13.mzXML Peak area,F_14.mzXML Peak area,F_15.mzXML Peak area,F_16.mzXML Peak area,F_17.mzXML Peak area
BioactivityCHIKV,NA,NA,68.0,1,4.0,1.00,3.0,19.00,8,16,41.0,140.0,17.00,10.50,5,57
1,270.279,1698,5690330.6,246596026,186949611.9,362958584.30,160968306.6,100851795.00,68548846,141101389,152605614.9,50568985.2,40253160.68,51162993.45,49859528,49792597
2,271.283,1699,569378.1,40396490,30710029.2,63895441.00,26317669.9,16091012.72,11523241,22836563,25351372.6,7873325.1,5872291.10,6356980.36,5143813,8061990
3,279.174,1413,3172924.3,0,0.0,0.00,0.0,93954.37,0,1538466,62451927.9,410657.7,41184.01,118658.96,0,0
4,280.264,1420,1821883.1,94393468,676012.6,97160.14,149605.1,199154.61,0,0,131789.8,2366320.9,0.00,25537.35,13143595,2932929


In [2]:
# Transpose and format column and row labels to follow the workflow below
# Change the the 'BioactivityCHIKV' variable in red below to the column row index name 
# corresponding to the bioassay results
tab <- t(in_tab[,-c(1:3)])
tab <- data.frame(Sample_name=sub("\\.mzXML Peak area", "", rownames(tab)), tab)
colnames(tab)[-1] <- c('BioactivityCHIKV', apply(in_tab[,2:3][-1,], 1, paste, collapse='_'))
rownames(tab) <- NULL

In [3]:
# Display the table 
tab[1:5,1:5]

Sample_name,BioactivityCHIKV,270.279_1698,271.283_1699,279.174_1413
Extract,68,5690331,569378.1,3172924
F_5,1,246596026,40396489.5,0
F_6,4,186949612,30710029.2,0
F_7,1,362958584,63895441.0,0
F_8,3,160968307,26317669.9,0


In [4]:
# Take out blank rows in the table
if(any(is.na(tab[,2]))) tab <- tab[!is.na(tab[,2]),]

In [5]:
# Add 1 to all to help scaling feature intensities and Normalize the features by TIC  
# 
tab2 <- tab
tab2[,-c(1:2)] <- t(apply(tab2[,-c(1:2)], 1, function(x) (x+1)/sum((x+1))))

In [6]:
# Calculate the correlation coefficient between a single feature and the bioactivity.
# Scale should help correlation - https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1534033/
cor.test(scale(tab2[,2])[,1], scale(tab2[,3])[,1])[c("estimate", "p.value")]

$estimate
       cor 
-0.3243776 

$p.value
[1] 0.2578494

In [7]:
# How to do for all features
ct <- t(sapply(3:ncol(tab2), function(x) unlist(cor.test(scale(tab2[,2])[,1], scale(tab2[,x])[,1])[c("estimate", "p.value")])))

In [8]:
# Show the dimensions of the features_quantificaton_matrix.csv
dim(tab2)
dim(ct)


[1]  14 589

[1] 587   2

In [9]:
# Create an output table with correlation coefficient value and p-value for every features

ct <- rbind(c("cor"," p_value"), c(0,0), ct)

tab3 <- rbind(t(ct),  as.matrix(tab2))
rownames(tab3) <- NULL
tab3[1:5, 1:5]
write.csv(tab3, "features_quantification_matrix_edited_with_correlation.csv", row.names=FALSE)

Sample_name,BioactivityCHIKV,270.279_1698,271.283_1699,279.174_1413
cor,0,-0.324377606725705,-0.318374649942669,0.11344978411662
p_value,0,0.257849356719369,0.267271199671765,0.699375059832539
Extract,68.0,0.0001391824,0.0000139267,7.760803e-05
F_5,1.0,0.1594079650,0.0261136500,6.464336e-10
F_6,4.0,0.0228397314,0.0037518603,1.221705e-10


In [10]:
# Tranpose the table for molecular networking mapping in Cytoscape
new = t(tab3)
colnames(new) = new[1,]
new = new[-1,]
new = cbind(0:(nrow(new)-1), rownames(new), new)
rownames(new) <- NULL
colnames(new)[1:2] <- c("shared name", "IDs")
new[1,1] <- ""
new[1:5,1:5]
write.csv(new, "features_quantification_matrix_transposed_with_correlation.csv", row.names=FALSE)

shared name,IDs,cor,p_value,Extract
,BioactivityCHIKV,0,0,68.0
1,270.279_1698,-0.324377606725705,0.257849356719369,0.0001391824
2,271.283_1699,-0.318374649942669,0.267271199671765,0.0000139267
3,279.174_1413,0.11344978411662,0.699375059832539,7.760803e-05
4,280.264_1420,-0.200702959079237,0.49145327312603,4.456230e-05


In [11]:
# Get the significant correlation coefficients for both cases (>0.05)
which(as.numeric(ct[-c(1,2),2])<0.05)

[1]  17  27  29  30  94  99 100 108 115 120 137 138 157 158 162 171 183 201 202
[20] 207 210 239 241 255 285 292 299 306 323 329 331 346 357 360 366 367 370 371
[39] 390 416 431 433 440 446 461 462 467 469 476 482 487 496 504 512 513 518 524
[58] 531 543 544 545 556

In [12]:
# Show the features ID with correlation coefficient
nm <- colnames(tab)
nm[-c(1:2)][as.numeric(ct[-c(1,2),2])<0.05]

[1] "383.222_1524" "401.232_1290" "401.232_1619" "401.232_1527" "563.296_1291"
 [6] "571.255_559"  "571.301_1719" "577.265_998"  "589.311_1520" "591.326_1626"
[11] "613.266_824"  "613.266_874"  "641.294_836"  "641.294_742"  "647.307_997" 
[16] "661.287_945"  "669.289_697"  "681.29_558"   "681.29_742"   "683.303_823" 
[21] "684.31_823"   "701.314_827"  "703.29_730"   "709.284_697"  "721.303_945" 
[26] "723.299_822"  "725.339_774"  "730.34_898"   "737.31_963"   "739.319_785" 
[31] "739.319_593"  "747.321_773"  "753.332_773"  "757.285_870"  "758.373_1047"
[36] "758.373_1257" "761.301_805"  "761.301_595"  "767.345_870"  "780.353_1045"
[41] "789.332_870"  "789.332_1014" "792.357_1201" "793.361_1211" "802.398_1055"
[46] "803.307_939"  "806.335_1158" "806.335_1061" "814.337_1192" "817.321_1105"
[51] "820.349_1210" "824.377_1038" "828.318_1067" "834.368_1269" "834.368_1463"
[56] "836.384_1150" "840.319_1120" "844.411_1331" "856.348_1463" "856.348_1245"
[61] "858.369_1147" "866.393_1330"

In [13]:
# Call the ID
which(p.adjust(as.numeric(ct[-c(1:2),2]), method = "bonferroni")<0.05)

[1]  27  30  94  99 108 115 120 137 138 157 162 171 183 201 207 210 239 241 255
[20] 285 299 331 346 357 360 366 367 371 390 416 431 433 446 461 462 469 487 496
[39] 504 513 518 524 543 544 545 556

In [14]:
# Features passing Bonferronii method
nm[-c(1:2)][which(p.adjust(as.numeric(ct[-c(1:2),2]), method = "bonferroni")<0.05)]


[1] "401.232_1290" "401.232_1527" "563.296_1291" "571.255_559"  "577.265_998" 
 [6] "589.311_1520" "591.326_1626" "613.266_824"  "613.266_874"  "641.294_836" 
[11] "647.307_997"  "661.287_945"  "669.289_697"  "681.29_558"   "683.303_823" 
[16] "684.31_823"   "701.314_827"  "703.29_730"   "709.284_697"  "721.303_945" 
[21] "725.339_774"  "739.319_593"  "747.321_773"  "753.332_773"  "757.285_870" 
[26] "758.373_1047" "758.373_1257" "761.301_595"  "767.345_870"  "780.353_1045"
[31] "789.332_870"  "789.332_1014" "793.361_1211" "802.398_1055" "803.307_939" 
[36] "806.335_1061" "820.349_1210" "824.377_1038" "828.318_1067" "834.368_1463"
[41] "836.384_1150" "840.319_1120" "856.348_1463" "856.348_1245" "858.369_1147"
[46] "866.393_1330"

In [15]:
# Prepare the new table
new <- cbind(new[,1:5], c(0, p.adjust(as.numeric(ct[-c(1:2),2]), method = "bonferroni")), new[,-c(1:5)])
colnames(new)[6] <- "p_value_corrected"
new[,1:10]

shared name,IDs,cor,p_value,Extract,p_value_corrected,F_5,F_6,F_7,F_8
,BioactivityCHIKV,0,0,68.0,0,1.0,4.0,1.0,3.0
1,270.279_1698,-0.324377606725705,0.257849356719369,0.0001391824,1,0.1594079650,0.0228397314,0.3139762067,0.0551088883
2,271.283_1699,-0.318374649942669,0.267271199671765,0.0000139267,1,0.0261136500,0.0037518603,0.0552725554,0.0090100816
3,279.174_1413,0.11344978411662,0.699375059832539,7.760803e-05,1,6.464336e-10,1.221705e-10,8.650469e-10,3.423586e-10
4,280.264_1420,-0.200702959079237,0.49145327312603,4.456230e-05,1,6.101911e-02,8.258893e-05,8.404894e-05,5.121893e-05
5,281.19_1031,-0.201330433729144,0.490071225054009,4.369353e-05,1,6.464336e-10,1.194904e-04,2.992650e-02,8.518229e-05
6,282.279_1660,-0.201928970163321,0.48875460960341,4.563084e-04,1,5.618968e-01,1.014960e-03,2.860098e-03,9.171837e-04
7,297.185_1652,-0.27121389090935,0.348283666308653,6.741464e-05,1,6.464336e-10,1.150745e-02,8.201229e-03,7.540400e-05
8,297.185_1556,-0.214874888224598,0.460688215436271,3.286022e-04,1,6.464336e-10,4.233761e-03,5.458206e-02,4.329827e-04
9,298.311_1968,-0.296642926666024,0.3030698225754,2.040302e-05,1,2.699247e-02,6.353464e-03,7.801129e-02,7.468284e-03


In [16]:
# Write the final table with corrected p_value
write.csv(new, "features_quantification_matrix_transposed_with_significant_correlation_pvalue_corrected.csv", row.names=FALSE)